In [ ]:
%pip install install git+https://github.com/henrikbulldog/xdbutils.git

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Cloning https://github.com/henrikbulldog/xdbutils.git to /tmp/pip-req-build-2c20rlh1
  Running command git clone --filter=blob:none --quiet https://github.com/henrikbulldog/xdbutils.git /tmp/pip-req-build-2c20rlh1
  Resolved https://github.com/henrikbulldog/xdbutils.git to commit 5e4d826adaf34ee857ba9250372af39724dad78d
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [ ]:
from xdbutils import XDBUtils

xdbutils = XDBUtils(spark, dbutils)

pipeline = xdbutils.create_dlt_batch_pipeline(
  source_system="testcdc",
  entity="employee",
  catalog="testing_dlt",
  tags={
    "data_owner": "Henrik Thomsen",
    "cost_center": "123456",
    "documentation": "https://github.com/henrikbulldog/xdbutils"
  },
  databricks_token=dbutils.secrets().get(scope="<scope>", key="<secret>")
)

In [ ]:
pipeline.raw_to_bronze(
  raw_base_path="dbfs:/FileStore/datalakehouse/raw",
  raw_format="json",
  expectations={
    "Valid EmpID": "EmpID IS NOT NULL",
    }
)

Name,Type
EmpID,bigint
Function,string
Name,string
Role,string
Updated,string
_rescued_data,string
_ingest_time,timestamp
_quarantined,boolean


In [ ]:
from pyspark.sql.functions import col

pipeline.bronze_to_silver(
  keys=["emp_id"],
  sequence_by="updated",
  parse=lambda df: df.select(
    col("EmpID").alias("emp_id"),
    col("Function").alias("function"),
    col("Name").alias("name"),
    col("Role").alias("role"),
    col("Updated").alias("updated"),
    )
  )

Name,Type
emp_id,bigint
function,string
name,string
role,string
updated,string


In [ ]:
from pyspark.sql.functions import *

pipeline.bronze_to_silver_track_changes(
  keys=["emp_id"],
  sequence_by="updated",
  parse=lambda df: df.select(
    col("EmpID").alias("emp_id"),
    col("Function").alias("function"),
    col("Name").alias("name"),
    col("Role").alias("role"),
    col("Updated").alias("updated"),
    )
  )

Name,Type
emp_id,bigint
function,string
name,string
role,string
updated,string


In [ ]:
pipeline.silver_to_gold(
  name="by_function",
  parse=lambda df: df.groupBy("function").agg(count("*").alias("total")),
  )

Name,Type
function,string
total,bigint


In [ ]:
pipeline.silver_to_gold(
  name="by_role",
  parse=lambda df: df.groupBy("role").agg(count("*").alias("total")),
  )

Name,Type
role,string
total,bigint
